# Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime

# fold path

In [2]:
raw_fold = 'D:/data/low_gas_vehicle/raw'
an_fold = 'D:/data/low_gas_vehicle/analysis'

# Load

## 등록정보

In [24]:
# 2m 40s
# car = pd.read_csv(os.path.join(ods_fold, 'ODS_CEG_CAR_MIG.csv'), low_memory=False, usecols=['EMIS_CRTCNO', 'FST_REG_DE', 'VHCL_ERSR_YN', 'VIN', 'SRCMNNO', 'EMIS_GRD'])
# 2023.07.08 기준(확인 : 어니컴 유덕수 대리)
car = pd.read_csv(os.path.join(raw_fold, 'STD_CEG_CAR_MIG(20230708).csv'), low_memory=False, usecols=['EXHST_GAS_CERT_NO', 'FRST_REG_YMD', 'VHCL_ERSR_YN', 'VIN', 'MANP_MNG_NO', 'EXHST_GAS_GRD_CD', 'VHCTY_CD', 'PURPS_CD2', 'VHCL_OWNR_CL_CD', 'BSPL_STDG_CD'])
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30625663 entries, 0 to 30625662
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   BSPL_STDG_CD       int64 
 1   EXHST_GAS_CERT_NO  object
 2   EXHST_GAS_GRD_CD   object
 3   FRST_REG_YMD       object
 4   VHCL_OWNR_CL_CD    object
 5   PURPS_CD2          object
 6   MANP_MNG_NO        object
 7   VHCL_ERSR_YN       object
 8   VHCTY_CD           object
 9   VIN                object
dtypes: int64(1), object(9)
memory usage: 2.3+ GB


In [122]:
cdict = {
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_ERSR_YN':'차량말소YN',
    'VIN':'차대번호',
    'MANP_MNG_NO':'제원관리번호',
    'EXHST_GAS_GRD_CD':'배출가스등급',
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도',
    'VHCL_OWNR_CL_CD':'사용자구분', 
    'BSPL_STDG_CD':'본거지법정동코드',
#     'OWNR_SE':'소유자구분',
#     'UPDT_DT':'수정일시',
#     'VHRNO':'차량등록번호',
#     'CRTN_DT':'생성일시',
#     'EMIS_INSP_EFCT_DE':'배출가스검사유효일자',
#     'EMIS_CRTCNO':'배출가스인증번호',
#     'NOW_MBERNO':'현재회원번호',
#     'ACQS_DE':'취득일자',
#     'ACQS_AMT':'취득금액',
#     'FST_MBERNO':'최초회원번호',
#     'VHRNO_NUM_INFO':'차량등록번호숫자정보',
#     'VHMNO':'차량관리번호',
#     'VHCL_MNFCT_DE':'차량제작일자',
#     'VHCL_YRIDNW':'차량연식',
#     'VHCL_ERSR_YN':'차량말소YN',
#     'VIN':'차대번호',
#     'LEM_KND':'저공해조치종류',
#     'LEM_STRCHG_YN':'저공해조치구조변경YN',
#     'LEM_YN':'저공해조치YN(미사용)',
#     'NOW_OWNR_NM':'미정',
#     'BSPL_ADRS':'미정',
#     'OWNR_ADRS':'미정',
#     'BSPL_ZIP':'미정',
#     'FST_OWNR_NM':'미정',
#     'RMK':'미정',
#     'USER_ID':'미정',
}
len(cdict)

10

In [123]:
carr = car.rename(columns=cdict)
carr.columns

Index(['본거지법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호'],
      dtype='object')

In [124]:
carr = carr.rename(columns={'본거지법정동코드':'법정동코드'})
carr.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호'],
      dtype='object')

## 제원정보

In [28]:
# 3s
# src = pd.read_csv(os.path.join(ods_fold, 'ODS_CEG_CAR_SRC_MIG.csv'), low_memory=False, usecols=['SRCMNNO', 'FUEL', 'VHCNM', 'VHCTY', 'VHCTY_TY'])
src = pd.read_csv(os.path.join(raw_fold, 'STD_CEG_CAR_SRC_MIG(20230708).csv'), low_memory=False, usecols=['MANP_MNG_NO', 'FUEL_CD', 'VHCNM', 'VHCTY_CL_CD', 'VHCTY_TY_CD2'])
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   FUEL_CD       545937 non-null  object
 1   MANP_MNG_NO   566615 non-null  object
 2   VHCNM         566544 non-null  object
 3   VHCTY_CL_CD   551140 non-null  object
 4   VHCTY_TY_CD2  553038 non-null  object
dtypes: object(5)
memory usage: 21.6+ MB


In [29]:
cdict = {
    'MANP_MNG_NO':'제원관리번호',
    'FUEL_CD':'연료',
    'VHCNM':'차명',
    # 'VHCTY_CD':'차종', 
    'VHCTY_CL_CD':'차종분류',
    'VHCTY_TY_CD2':'차종유형', 
#     'UPDT_DT':'수정일시',
#     'CNFRM_DE':'승인일자',
#     'DELE_YN':'삭제YN', 
#     'CRTN_DT':'생성일시', 
#     'TOTL_WGHT':'총중량',
#     'VHCTY_TY':'차종유형(소분류)', 
#     'VHCTY_CL':'차종분류',
#     'VHCTY':'차종', 
#     'VHCNM':'차명',
#     'VHCL_HG':'차량높이',
#     'VHCL_WH':'차량너비',
#     'VHCL_LT':'차량길이',
#     'DRV_FRM':'구동형식',
#     'MKR_NM':'제작사명',
#     'SRCMNNO':'제원관리번호',
#     'CRYNG_WGHT':'적재중량',
#     'CAR_FRM':'자동차형식',
#     'EMPV_WGHT':'공차중량',
#     'PLOR_NM':'원산지명',
#     'FUEL':'연료',
#     'EGINTY':'엔진형식',
#     'EGINPWR':'엔진출력',
#     'TKCAR_NMPR':'승차인원',
#     'DSPLVL':'배기량',
#     'GEARBOX_KND':'변속기종류',
#     'REPCAR_NM':'미정',
#     'RMK':'미정',
#     'USER_ID':'미정',
}
srcr = src.rename(columns=cdict)
srcr.columns

Index(['연료', '제원관리번호', '차명', '차종분류', '차종유형'], dtype='object')

## 본거지법정동코드

In [30]:
# 3s
code = pd.read_csv(os.path.join(raw_fold, 'STD_BJCD_INFO(20230904).csv'), low_memory=False, usecols=['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM', 'ABL_YN'])
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   ABL_YN        47916 non-null  object
 2   STDG_CTPV_NM  47916 non-null  object
 3   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [31]:
cdict = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    # 'STDG_CTPV_CD':'시도코드', 
    # 'STDG_SGG_CD':'시군구코드', 
    'ABL_YN':'폐지여부', 
}
coder = code.rename(columns=cdict)
coder.columns

Index(['법정동코드', '폐지여부', '시도', '시군구'], dtype='object')

In [32]:
coder['폐지여부'].value_counts(dropna=False)

폐지여부
폐지    27263
존재    20653
Name: count, dtype: int64

In [33]:
coder = coder[coder['폐지여부'] == '존재'].reset_index(drop=True)
coder.shape

(20653, 4)

## 본거지 법정동 코드(대기관리권역정보)

In [34]:
# 3s
air = pd.read_excel(os.path.join(raw_fold, '법정동코드_2022.10.14(수정2023.05.04).xlsx'), usecols=['시도', '시구군', '대기관리권역', '대기관리권역YN'])
air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   시도        46342 non-null  object
 1   시구군       46319 non-null  object
 2   대기관리권역    46342 non-null  object
 3   대기관리권역YN  46337 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [35]:
air = air.rename(columns={'시구군':'시군구'})
air.columns

Index(['시도', '시군구', '대기관리권역', '대기관리권역YN'], dtype='object')

In [36]:
air[air['시도'] == '강원도'].shape

(2846, 4)

In [37]:
air.loc[air['시도'] == '강원도', '시도'] = '강원특별자치도'

In [38]:
air[air['시도'] == '강원도'].shape

(0, 4)

# Preprocessing

## 등록정보

In [116]:
carr.shape

(30625663, 10)

In [125]:
# 중복 차대번호 제거
# 1m 48s
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(30610548, 10)

In [126]:
carr['배출가스등급'].unique()

array([nan, 'A0503', 'A0502', 'A0501', 'A0505', 'A0504'], dtype=object)

In [127]:
carr['배출가스등급'].value_counts(dropna=False)

배출가스등급
A0502    12142816
A0503     9897390
A0501     3293520
A0505     2313549
A0504     1905402
NaN       1057871
Name: count, dtype: int64

In [128]:
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'] = carr['배출가스등급'].fillna('X')
carr['배출가스등급'].unique()

array(['X', '3', '2', '1', '5', '4'], dtype=object)

In [129]:
carr['배출가스등급'].value_counts(dropna=False)

배출가스등급
2    12142816
3     9897390
1     3293520
5     2313549
4     1905402
X     1057871
Name: count, dtype: int64

In [130]:
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '특수', '승합'], dtype=object)

In [131]:
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

In [132]:
# ## 소유자구분 코드 변환
# ownr_dict = {
#     'A27B':'사업자', 
#     'A27F':'외국인', 
#     'A27L':'법인', 
#     'A27O':'기타', 
#     'A27R':'주민', 
# }
# carr['소유자구분'] = carr['소유자구분'].replace(ownr_dict)
# carr['소유자구분'].unique()

In [133]:
carr['법정동코드'].dtype

dtype('int64')

## 제원정보

In [52]:
srcr.shape

(566615, 5)

In [53]:
# srcr['차종'].unique()

In [54]:
# srcr['차종'].value_counts(dropna=False)

In [55]:
# ## 차종 코드 변환
# cd_dict = {
#     'A31M':'이륜', 
#     'A31P':'승용', 
#     'A31S':'특수', 
#     'A31T':'화물', 
#     'A31V':'승합'
# }
# srcr['차종'] = srcr['차종'].replace(cd_dict)
# srcr['차종'].unique()

In [56]:
# srcr['차종'].value_counts(dropna=False)

In [57]:
srcr['차종분류'].unique()

array(['A2826', 'A2814', 'A2825', 'A2819', 'A2821', 'A2824', 'A2811',
       'A2820', 'A2810', 'A2809', 'A2816', 'A2815', 'A2823', 'A2822',
       'A2813', 'A2812', nan], dtype=object)

In [58]:
## 차종분류 코드 변환
cl_dict = {
    'A2801':'견인', 
    'A2802':'구난', 
    'A2803':'기타', 
    'A2804':'다목적',
    'A2805':'덤프',
    'A2806':'밴',
    'A2807':'승용겸화물',
    'A2808':'일반',
    'A2809':'일반형-카고',
    'A2810':'일반형-픽업',
    'A2811':'특수',
    'A2812':'특수용도형',
    'A2813':'특수용도형-탱크로리',
    'A2814':'특수용도형-특수용도형',
    'A2815':'특수용도형-피견인',
    'A2816':'특수작업형',
    'A2818':'특수', 
    'A2819':'다목적', 
    'A2820':'덤프', 
    'A2821':'기타', 
    'A2822':'견인', 
    'A2823':'구난', 
    'A2824':'밴', 
    'A2825':'승용겸화물', 
    'A2826':'일반', 

}
srcr['차종분류'] = srcr['차종분류'].replace(cl_dict)
srcr['차종분류'].unique()

array(['일반', '특수용도형-특수용도형', '승용겸화물', '다목적', '기타', '밴', '특수', '덤프',
       '일반형-픽업', '일반형-카고', '특수작업형', '특수용도형-피견인', '구난', '견인', '특수용도형-탱크로리',
       '특수용도형', nan], dtype=object)

In [59]:
srcr['차종유형'].unique()

array(['A30L', 'A30M', 'A30S', 'A30C', nan], dtype=object)

In [60]:
srcr['차종유형'].value_counts(dropna=False)

차종유형
A30L    356019
A30M    132073
A30S     55204
NaN      13577
A30C      9742
Name: count, dtype: int64

In [61]:
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

In [62]:
srcr['차종유형'].value_counts(dropna=False)

차종유형
대형     356019
중형     132073
소형      55204
NaN     13577
경형       9742
Name: count, dtype: int64

In [63]:
srcr['연료'].unique()

array(['A90GS', 'A91DS', 'A94OT', 'A92LP', nan, 'A90GH', 'A92CN', 'A91KS',
       'A93SH', 'A91DH', 'A93EV', 'A90AC', 'A92LH', 'A93HD', 'A92LN',
       'A92CH'], dtype=object)

In [64]:
srcr['연료'].value_counts(dropna=False)

연료
A91DS    284477
A90GS    218330
A94OT     23793
NaN       20678
A92CN      7380
A92LP      4939
A90GH      4149
A93EV      2212
A91DH       361
A92CH       105
A93HD        98
A92LN        25
A91KS        20
A92LH        19
A90AC        17
A93SH        12
Name: count, dtype: int64

In [65]:
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG(액화석유가스)', nan, '휘발유 하이브리드',
       'CNG(압축천연가스)', '등유', '태양열', '경유 하이브리드', '전기', '알코올', 'LPG 하이브리드',
       '수소', 'LNG(액화천연가스)', 'CNG 하이브리드'], dtype=object)

In [66]:
srcr['연료'].value_counts(dropna=False)

연료
경유             284477
휘발유            218330
기타연료            23793
NaN             20678
CNG(압축천연가스)      7380
LPG(액화석유가스)      4939
휘발유 하이브리드        4149
전기               2212
경유 하이브리드          361
CNG 하이브리드         105
수소                 98
LNG(액화천연가스)        25
등유                 20
LPG 하이브리드          19
알코올                17
태양열                12
Name: count, dtype: int64

## 법정동 코드 대기관리권역 정보 추가

In [67]:
air.shape, air.drop_duplicates(['시도', '시군구']).shape

((46342, 4), (443, 4))

In [68]:
air['대기관리권역'].value_counts(dropna=False)

대기관리권역
동남권    17032
중부권    12327
수도권     8225
남부권     5339
그 외     3419
Name: count, dtype: int64

In [69]:
air.sort_values(['대기관리권역']).tail()

,시도,시군구,대기관리권역,대기관리권역YN
18782,충청남도,논산시,중부권,Y
18783,충청남도,논산시,중부권,Y
18784,충청남도,논산시,중부권,Y
18726,충청남도,논산시,중부권,Y
23170,전라북도,정읍시,중부권,N


In [70]:
air1 = air.sort_values(['대기관리권역']).drop_duplicates(['시도', '시군구']).reset_index(drop=True)
air1.shape

(443, 4)

In [71]:
air1.head()

,시도,시군구,대기관리권역,대기관리권역YN
0,제주특별자치도,서귀포시,그 외,N
1,강원특별자치도,홍천군,그 외,N
2,강원특별자치도,춘천군,그 외,N
3,강원특별자치도,춘성군,그 외,N
4,강원특별자치도,횡성군,그 외,N


In [72]:
air1['대기관리권역'].unique()

array(['그 외', '남부권', '동남권', '수도권', '중부권'], dtype=object)

In [73]:
coder.columns

Index(['법정동코드', '폐지여부', '시도', '시군구'], dtype='object')

In [74]:
coder.shape

(20653, 4)

In [75]:
air['시도'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도',
       '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [76]:
coder['시도'].unique()

array(['경기도', '대전광역시', '울산광역시', '대구광역시', '세종특별자치시', '인천광역시', '광주광역시',
       '충청북도', '충청남도', '서울특별시', '부산광역시', '전라북도', '전라남도', '경상북도', '경상남도',
       '제주특별자치도', '강원특별자치도'], dtype=object)

In [77]:
cd1 = coder.merge(air1, on=['시도', '시군구'], how='left')
cd1.shape

(20653, 6)

In [78]:
cd1.head()

,법정동코드,폐지여부,시도,시군구,대기관리권역,대기관리권역YN
0,4119011200,존재,경기도,부천시,수도권,Y
1,4119011300,존재,경기도,부천시,수도권,Y
2,4119011400,존재,경기도,부천시,수도권,Y
3,4119011500,존재,경기도,부천시,수도권,Y
4,4119011600,존재,경기도,부천시,수도권,Y


In [79]:
cd1['대기관리권역'].value_counts(dropna=False)

대기관리권역
동남권    6520
중부권    6135
남부권    3205
수도권    2992
그 외    1688
NaN     113
Name: count, dtype: int64

In [80]:
cd1.loc[cd1['대기관리권역'].isnull(), '시도'].unique()

array(['대구광역시', '부산광역시'], dtype=object)

In [81]:
air.shape, air1.shape, air.drop_duplicates('시도').shape

((46342, 4), (443, 4), (17, 4))

In [82]:
air2 = air.drop_duplicates('시도').reset_index(drop=True)
air2.shape

(17, 4)

In [83]:
air2['시도'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전라북도', '전라남도',
       '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [84]:
cd1.shape

(20653, 6)

In [85]:
cd1y = cd1[cd1['대기관리권역'].isnull() == False]
cd1n = cd1[cd1['대기관리권역'].isnull() == True]
cd1y.shape[0] + cd1n.shape[0]

20653

In [86]:
cd1n.shape

(113, 6)

In [87]:
cd1n.columns

Index(['법정동코드', '폐지여부', '시도', '시군구', '대기관리권역', '대기관리권역YN'], dtype='object')

In [88]:
air2.head()

,시도,시군구,대기관리권역,대기관리권역YN
0,서울특별시,NaN,수도권,Y
1,부산광역시,NaN,동남권,Y
2,대구광역시,NaN,동남권,Y
3,인천광역시,NaN,수도권,Y
4,광주광역시,NaN,남부권,Y


In [89]:
cd1n = cd1n.drop(['시군구', '대기관리권역', '대기관리권역YN'], axis=1)
cd1n.shape

(113, 3)

In [90]:
cd1nm = cd1n.merge(air2, on='시도', how='left')
cd1nm.shape

(113, 6)

In [91]:
cd1nm['대기관리권역'].value_counts(dropna=False)

대기관리권역
동남권    113
Name: count, dtype: int64

In [92]:
cd = pd.concat([cd1y, cd1nm], ignore_index=True)
cd.shape

(20653, 6)

In [93]:
cd1.shape

(20653, 6)

In [94]:
cd['대기관리권역'].value_counts(dropna=False)

대기관리권역
동남권    6633
중부권    6135
남부권    3205
수도권    2992
그 외    1688
Name: count, dtype: int64

### [출력] 법정동코드(2023.07.08기준)

In [95]:
# # 0s
# cd.to_csv(os.path.join(raw_fold, '법정동코드(2023.07.08).csv'), index=False)

### [로드] 법정동코드(2023.07.08기준)

In [96]:
# # 0s
# cd = pd.read_csv(os.path.join(raw_fold, '법정동코드(2023.07.08).csv'))
# cd.info()

## 등록&제원 병합

In [134]:
carr.shape

(30610548, 10)

In [135]:
# 8.6s
cs = carr.merge(srcr, on='제원관리번호', how='left')
cs.shape

(30610548, 14)

In [136]:
cs.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호', '연료', '차명', '차종분류', '차종유형'],
      dtype='object')

### [출력] 등록&제원(중복차대제거)(2023.07.08 기준)
<!-- ### [출력] 등록&제원(중복차대제거)(2022.12.28 기준) -->
- 중복 차대번호 제거

In [137]:
# 2m 20s
cs.to_csv(os.path.join(raw_fold, '등록&제원(중복차대제거)(2023.07.08).csv'), index=False)

# [로드] 등록&제원(중복차대제거)(2023.07.08 기준)
<!-- # [로드] 등록&제원(중복차대제거)(2022.12.28 기준) -->

In [101]:
# # 7m
# # cs = pd.read_csv(os.path.join(raw_fold, '[ODS]등록정보&제원정보(말소제거)(2022.12.28).csv'), low_memory=False, encoding='cp949')

# # 1m 30s
# cs = pd.read_csv(os.path.join(raw_fold, '등록&제원(중복차대제거)(2023.07.08).csv'), low_memory=False)
# cs.info()

## 등록&제원&법정동코드

In [138]:
cs.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호', '연료', '차명', '차종분류', '차종유형'],
      dtype='object')

In [139]:
cs['법정동코드'].dtype

dtype('int64')

In [140]:
cs['법정동코드'] = cs['법정동코드'].astype('str')
cs['법정동코드'] = cs['법정동코드'].str[:5] + '00000'
cs['법정동코드'] = pd.to_numeric(cs['법정동코드'], errors='coerce')
cs['법정동코드'].dtype

dtype('int64')

In [141]:
## 법정동코드 수정
# 4119700000 경기도 부천시 소사구 폐지 -> 경기도 부천시 4119000000 
# 4119900000 경기도 부천시 오정구 폐지 -> 경기도 부천시 4119000000
# 2817000000 인천광역시 남구 존재 -> 인천광역시 미추홀구 2817700000
# 4371000000 충청북도 청원군 폐지 -> 충청북도 청주시 4311000000
# 4119500000 경기도 부천시 원미구 폐지 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 폐지 -> 경기도 여주시 4167000000

cs.loc[cs['법정동코드'] == 4119700000, '법정동코드'] = 4119000000
cs.loc[cs['법정동코드'] == 4119900000, '법정동코드'] = 4119000000
cs.loc[cs['법정동코드'] == 2817000000, '법정동코드'] = 2817700000
cs.loc[cs['법정동코드'] == 4371000000, '법정동코드'] = 4311000000
cs.loc[cs['법정동코드'] == 4119500000, '법정동코드'] = 4119000000
cs.loc[cs['법정동코드'] == 4173000000, '법정동코드'] = 4167000000

In [142]:
cd.columns

Index(['법정동코드', '폐지여부', '시도', '시군구', '대기관리권역', '대기관리권역YN'], dtype='object')

In [143]:
cs.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호', '연료', '차명', '차종분류', '차종유형'],
      dtype='object')

In [145]:
cs.shape

(30610548, 14)

In [361]:
# 7s
df = cs.merge(cd[['법정동코드', '시도', '대기관리권역', '대기관리권역YN']], on='법정동코드', how='left')
df.shape

(30610548, 17)

In [362]:
df.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호', '연료', '차명', '차종분류', '차종유형', '시도', '대기관리권역',
       '대기관리권역YN'],
      dtype='object')

In [363]:
df['대기관리권역'].isnull().sum()

0

In [364]:
df.loc[df['대기관리권역'].isnull(), '법정동코드'].unique()

array([], dtype=int64)

### [출력] 등록&제원&법정동코드

In [189]:
# 3m 22s
df.to_csv(os.path.join(raw_fold, '등록&제원&법정동(2023.07.08).csv'), index=False)

## 연료 수정

In [365]:
df['연료'].unique()

array(['휘발유', '경유', '휘발유 하이브리드', '전기', 'LPG(액화석유가스)', '경유 하이브리드', '기타연료',
       nan, '수소', 'CNG(압축천연가스)', 'LNG(액화천연가스)', 'LPG 하이브리드', '태양열', '알코올',
       'CNG 하이브리드', '등유'], dtype=object)

In [366]:
# 5s
df['연료'] = df['연료'].replace({'LPG(액화석유가스)':'LPG', 'CNG(압축천연가스)':'CNG', 'LNG(액화천연가스)':'LNG'})
df['연료'].unique()

array(['휘발유', '경유', '휘발유 하이브리드', '전기', 'LPG', '경유 하이브리드', '기타연료', nan,
       '수소', 'CNG', 'LNG', 'LPG 하이브리드', '태양열', '알코올', 'CNG 하이브리드', '등유'],
      dtype=object)

## 말소제거

In [367]:
df['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25795280
Y     4815268
Name: count, dtype: int64

In [368]:
df.shape

(30610548, 17)

In [369]:
# 18s
dfn = df[df['차량말소YN'] =='N'].reset_index(drop=True)
dfn.shape

(25795280, 17)

In [370]:
# 26s
len(dfn['차대번호'].unique())

25795280

# Analysis

## 1\. 등급별 연료별 차량 현황

In [371]:
dfn['배출가스등급'].unique()

array(['X', '3', '2', '1', '5', '4'], dtype=object)

In [372]:
dfn['배출가스등급'].value_counts(dropna=False)

배출가스등급
2    10967291
3     8293916
1     3142066
4     1390279
X     1027741
5      973987
Name: count, dtype: int64

In [373]:
dfn.columns

Index(['법정동코드', '배출가스인증번호', '배출가스등급', '최초등록일자', '사용자구분', '용도', '제원관리번호',
       '차량말소YN', '차종', '차대번호', '연료', '차명', '차종분류', '차종유형', '시도', '대기관리권역',
       '대기관리권역YN'],
      dtype='object')

In [374]:
carname_fuel_dict = {
    'HYUNDAI PRESTO':'휘발유',
    'TOYOTA CAMRY하이브리드':'휘발유 하이브리드',
    '구급차(도요다)':'경유',
    '대우11톤카고':'경유',
    '대우8톤카고':'경유',
    '도요다웨곤엠브란스':'경유',
    '도요다코로나':'휘발유',
    '도요다크라운로얄싸롱':'휘발유',
    '도요타 프리우스(하이브리드)':'휘발유 하이브리드',
    '로얄살롱':'휘발유',
    '로얄살롱2.0AT':'휘발유',
    '르망':'휘발유',
    '르망펜타-5':'휘발유',
    '맥스디젤삼방':'경유',
    '머큐리 세이블':'경유',
    '벤즈 230E':'휘발유',
    '벤즈230E':'휘발유',
    '복사14':'휘발유',
    '브리샤2':'휘발유',
    '새한버스':'경유',
    '스텔라':'휘발유',
    '스텔라1.6CXL':'휘발유',
    '스텔라오토매틱':'휘발유',
    '아시아AM707중형버스':'경유',
    '에어스트림탐지견수송트레일러':'무동력',
    '엑셀':'휘발유',
    '인타담프':'경유',
    '일산 육륜':"경유",
    '캐피탈':'휘발유',
    '코티나마크IV':'휘발유',
    '포니2':'휘발유',
    '프레스토오토매틱':'휘발유',
    '현대리어엔진버스':'경유',
    '홍성캠핑트레일러':'무동력',
    'BENZ S500':'휘발유',
    'BMW':'휘발유',
    'BMW 3-SERIES':'휘발유',
    'BMW 318I':'휘발유',
    'BMW 325CI':'휘발유',
    'BMW 325I':'휘발유',
    'BMW 325i':'휘발유',
    'BMW 328I':'휘발유',
    'BMW 525I':'휘발유',
    'BMW 528I':'휘발유',
    'BMW 530I':'휘발유',
    'BMW 5SERIES':'휘발유',
    'BMW 750Li':'휘발유',
    'BMW X3':'휘발유',
    'BMW X5':'휘발유',
    'BMW Z3':'휘발유',
    'BMW530I':'휘발유',
    'BUICK CENTURY':'휘발유',
    'BUICK LESABRE':'휘발유',
    'BUICK TERRAZA':'휘발유',
    'CADILLAC CTS':'휘발유',
    'CHEVROLET IMPALA':'휘발유',
    'CHEVROLET PRIZM':'휘발유',
    'CHRYSLER':'휘발유',
    'CHRYSLER 300 LTD':'휘발유',
    'CHRYSLER 300C':'경유',
    'CHRYSLER HEMI AWD':'휘발유',
    'CHRYSLER PT CRUISER':'휘발유',
    'CHRYSLER TOWN & COUNTRY(7)':'휘발유',
    'CIELO':'휘발유',
    'CORSICA':'휘발유',
    'COUGAR':'휘발유',
    'DODGE CALIBER':'휘발유',
    'DODGE CARAVAN':'휘발유',
    'DODGE CHARGER':'휘발유',
    'DODGE DAKOTA':'휘발유',
    'DODGE DURANGO':'휘발유',
    'DODGE GRAND CARAVAN':'휘발유',
    'DODGE NEON SXT':'휘발유',
    'EF소나타':'휘발유',
    'ESCORT':'휘발유',
    'FIREBIRD':'휘발유',
    'FORD ESCAPE':'휘발유',
    'FORD EXPEDITION':'휘발유',
    'FORD EXPEDITION(8)':'휘발유',
    'FORD EXPLORER':'휘발유',
    'FORD FOCUS':'휘발유',
    'FORD FREESTYLE(7)':'휘발유',
    'FORD FUSION':'휘발유',
    'FORD MUSTANG':'휘발유',
    'FORD RANGER XLT':'경유',
    'FORD WINDSTAR':'휘발유',
    'G.M.C':'휘발유',
    'G20':'휘발유',
    'GMC':'휘발유',
    'GMC JIMMY':'휘발유',
    'GMC SIERRA':'휘발유',
    'HONDA ACCORD':'휘발유',
    'HONDA CR-V':'휘발유',
    'HONDA CR-V EX':'휘발유',
    'HONDA PILOT':'휘발유',
    'HYUNDAI SONATA':'휘발유',
    'HYUNDAI/SONATA':'휘발유',
    'INFINITI G37S':'휘발유',
    'JAGUAR X-TYPE':'휘발유',
    'JEEP GRAND CHEROKEE':'휘발유',
    'JEEP LIBERTY':'휘발유',
    'JEEP PATRIOT':"경유",
    'KIA OPTIMA':'휘발유',
    'KIA SEDONA':'휘발유',
    'LEXUS ES330':'휘발유',
    'LEXUS ES350':'휘발유',
    'LEXUS LS430':'휘발유',
    'LEXUS RS330':'휘발유',
    'LEXUS RX330':'휘발유',
    'LINCOLN MKX':'휘발유',
    'MAZDA MAZDA6':'휘발유',
    'MAZDA SENTA':'휘발유',
    'MERCEDES BENZ':'휘발유',
    'MERCEDES-BENZ':'휘발유',
    'MERCEDES-BENZ CLS 550':'경유',
    'MERCEDES-BENZ E-350':'휘발유',
    'MERCEDES-BENZ E-CLASS':'휘발유',
    'MERCEDES-BENZ R-CLASS 3500':'경유',
    'MERCEDES-BENZ S 500':'휘발유',
    'MERCEDES-BENZ S65':'경유',
    'MERCURY MILAN':'휘발유',
    'NEW 그랜저XG':'휘발유',
    'NISSAN MURANO':'휘발유',
    'NISSAN ROGUE 3.5':'휘발유',
    'NISSAN XTERRA':'휘발유',
    'OLDSMOBILE BRAVADA':'휘발유',
    'OLDSMOBILESILHOUET':'휘발유',
    'PLYMOUTH':'휘발유',
    'PONTIAC SUNBIRD':'휘발유',
    'PONTIAC VIBE':'휘발유',
    'PONTIAC VIBE BASE':'휘발유',
    'PORSCHE':'휘발유',
    'ROYALE':'휘발유',
    'ROYALSALON':'휘발유',
    'RX7':'휘발유',
    'SANTAFE':'휘발유',
    'SEPHIA':'휘발유',
    'SILHOUET':'휘발유',
    'SPORTAGE':'휘발유',
    'STAREX':'휘발유',
    'SUNDANCE':'휘발유',
    'TAURUS':'휘발유',
    'TOYOTA 4-RUNNER':'경유',
    'TOYOTA AVALON':'휘발유',
    'TOYOTA COROLLA':'휘발유',
    'TOYOTA ESTIIM':'휘발유',
    'TOYOTA MATRIX':'휘발유',
    'TOYOTA SIENNA':'휘발유',
    'TOYOTA SR-5':'휘발유',
    'VISION':'휘발유',
    'VOLKSWAGEN JETTA':'휘발유',
    'VOLVO 7G1':'휘발유',
    'VOLVO 960':'휘발유',
    'VOLVO S60':'휘발유',
    'VOLVO S80':'휘발유',
    'VOLVO T5':'휘발유',
    'VOLVO XC60':'휘발유',
    'VOLVO XC90':'휘발유',
    '그랜저2.0':'휘발유',
    '그랜저XG300L':'휘발유',
    '기아마스타':'경유',
    '뉴복사담프':'경유',
    '뉴비틀2.0':'휘발유',
    '닛산실비아2.0':'휘발유',
    '닛산씨마그로리아':'휘발유',
    '다임24KL에틸렌글리콜트레일러':'무동력',
    '다찌람밴':'휘발유',
    '대우11톤카고트럭':'경유',
    '대흥저상트레일러':'무동력',
    '도오다':'휘발유',
    '도요다MR2':'휘발유',
    '도요다세라':'휘발유',
    '도요다슈프라':'휘발유',
    '도요다크라운':'휘발유',
    '라비타(LAVITA)':'휘발유',
    '로얄살롱':'휘발유',
    '르망':'휘발유',
    '링컨MARK7':'경유',
    '벤츠 C300':'휘발유',
    '벤츠190E':'휘발유',
    '벤츠C280':'휘발유',
    '벤츠E230':'휘발유',
    '벤츠S320':'휘발유',
    '블루버드':'휘발유',
    '사파리':'휘발유',
    '새턴':'휘발유',
    '새한2.5톤카고':'경유',
    '셀리카-GT':'휘발유',
    '소나타':'휘발유',
    '소나타1800오토매틱':'휘발유',
    '수프라':'휘발유',
    '시보레':'휘발유',
    '시보레카베리아':'휘발유',
    '쏘나타':'휘발유',
    '아큐라':'휘발유',
    '에스페로':'휘발유',
    '엑셀운전교습용차량':'휘발유',
    '원흥식 트럭':'경유',
    '제설차':'경유',
    '지프':'경유',
    '체로키':'휘발유',
    '캠리':'휘발유',
    '코르시카':'휘발유',
    '코우거':'휘발유',
    '코티나디럭스':'휘발유',
    '크로마터보i.e':'휘발유',
    '티코(TICO)':'휘발유',
    '포니':'휘발유',
    '포니2-L.P.G':'LPG(액화석유가스)',
    '포니2-LPG':'LPG(액화석유가스)',
    '포르쉐':'휘발유',
    '프레스토오토메틱':'휘발유',
    '프리무스':'휘발유 하이브리드',
    '한원40A콤비콘테이너트레일러':'무동력',
    '한원40피트구즈넥트레일러':'무동력',
    '현대3톤트럭':'경유',
    '현대포터':'경유',
    '혼다 CR-V':'경유',
}

In [375]:
len(carname_fuel_dict)

216

In [376]:
dfn['연료'].value_counts(dropna=False)

연료
휘발유          12272647
경유            9627362
LPG           1827225
휘발유 하이브리드     1320061
전기             478853
NaN             75672
기타연료            73912
경유 하이브리드        44519
수소              32909
CNG             28731
LPG 하이브리드       13064
CNG 하이브리드         292
LNG                26
알코올                 3
태양열                 2
등유                  2
Name: count, dtype: int64

#### 차명, 연료 매칭

In [377]:
# # about 5m 
# for k, v in tqdm(carname_fuel_dict.items()):
#     dfn.loc[(dfn['차명'] == k) & ((dfn['연료'].isnull() == True) | (dfn['연료'].isnull() == '기타연료')), '연료'] = v

In [378]:
dfn['연료'].value_counts(dropna=False)

연료
휘발유          12272647
경유            9627362
LPG           1827225
휘발유 하이브리드     1320061
전기             478853
NaN             75672
기타연료            73912
경유 하이브리드        44519
수소              32909
CNG             28731
LPG 하이브리드       13064
CNG 하이브리드         292
LNG                26
알코올                 3
태양열                 2
등유                  2
Name: count, dtype: int64

In [379]:
# # 13s
# dfn.loc[(dfn['배출가스등급'] == '2') | (dfn['배출가스등급'] == 'T2'), '배출가스등급'] = '2'
# dfn.loc[(dfn['배출가스등급'] == '3') | (dfn['배출가스등급'] == 'T3'), '배출가스등급'] = '3'
# dfn.loc[(dfn['배출가스등급'] == '4') | (dfn['배출가스등급'] == 'T4'), '배출가스등급'] = '4'
# dfn.loc[(dfn['배출가스등급'] == '5') | (dfn['배출가스등급'] == 'T5'), '배출가스등급'] = '5'
# dfn.loc[(dfn['배출가스등급'] == 'X') | (dfn['배출가스등급'].isnull() == True), '배출가스등급'] = 'X'

In [407]:
dfn['배출가스등급'].unique()

array(['X', '3', '2', '1', '5', '4'], dtype=object)

In [408]:
dfn['배출가스등급'].value_counts(dropna=False)

배출가스등급
2    10967291
3     8293916
1     3142066
4     1390279
X     1027741
5      973987
Name: count, dtype: int64

In [409]:
dfn['연료'].unique()

array(['휘발유', '경유', '휘발유 하이브리드', '전기', 'LPG', '경유 하이브리드', '기타연료', nan,
       '수소', 'CNG', 'LNG', 'LPG 하이브리드', '태양열', '알코올', 'CNG 하이브리드', '등유'],
      dtype=object)

In [410]:
dfn.shape

(25795280, 17)

In [411]:
# X등급 제외
# 연료 : 등유, 태양열 제외, 빈값 제외
dfg = dfn[(dfn['배출가스등급'] != 'X') & ((dfn['연료'] != '등유') & (dfn['연료'] != '태양열') & (dfn['연료'].isnull() == False))].reset_index(drop=True)
dfg.shape

(24767342, 17)

In [412]:
# 21s
grp1 = dfg.groupby(['배출가스등급', '연료'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp1

배출가스등급,연료,1,2,3,4,5
0,CNG,NaN,22511.0,4444.0,1.0,7.0
1,CNG 하이브리드,275.0,17.0,NaN,NaN,NaN
2,LNG,NaN,2.0,NaN,NaN,NaN
3,LPG,454141.0,1132058.0,194059.0,24355.0,430.0
4,LPG 하이브리드,13062.0,NaN,NaN,NaN,NaN
5,경유,NaN,NaN,7343084.0,1052801.0,943899.0
6,경유 하이브리드,NaN,NaN,18376.0,NaN,NaN
7,기타연료,NaN,NaN,5.0,34.0,20.0
8,수소,32909.0,NaN,NaN,NaN,NaN
9,알코올,NaN,NaN,NaN,1.0,1.0


In [413]:
grp1.shape

(13, 6)

In [414]:
grp1['합계'] = grp1.iloc[:, 1:].sum(axis=1)
grp1.loc[grp1.shape[0]] = ['합계'] + list(grp1.iloc[:, 1:].sum().values)
grp1

배출가스등급,연료,1,2,3,4,5,합계
0,CNG,NaN,22511.0,4444.0,1.0,7.0,26963.0
1,CNG 하이브리드,275.0,17.0,NaN,NaN,NaN,292.0
2,LNG,NaN,2.0,NaN,NaN,NaN,2.0
3,LPG,454141.0,1132058.0,194059.0,24355.0,430.0,1805043.0
4,LPG 하이브리드,13062.0,NaN,NaN,NaN,NaN,13062.0
5,경유,NaN,NaN,7343084.0,1052801.0,943899.0,9339784.0
6,경유 하이브리드,NaN,NaN,18376.0,NaN,NaN,18376.0
7,기타연료,NaN,NaN,5.0,34.0,20.0,59.0
8,수소,32909.0,NaN,NaN,NaN,NaN,32909.0
9,알코올,NaN,NaN,NaN,1.0,1.0,2.0


In [415]:
idf1 = pd.DataFrame({'연료':['휘발유', '휘발유 하이브리드', '경유', '경유 하이브리드', 'LPG', 'LPG 하이브리드', 'LNG', 'CNG', 'CNG 하이브리드', '전기', '수소', '알코올', '기타연료', '합계']})
idf1

,연료
0,휘발유
1,휘발유 하이브리드
2,경유
3,경유 하이브리드
4,LPG
5,LPG 하이브리드
6,LNG
7,CNG
8,CNG 하이브리드
9,전기


In [416]:
base1 = idf1.merge(grp1, on='연료', how='left')
base1

,연료,1,2,3,4,5,합계
0,휘발유,1082492.0,9726237.0,733843.0,313038.0,29610.0,11885220.0
1,휘발유 하이브리드,1080390.0,86443.0,NaN,NaN,NaN,1166833.0
2,경유,NaN,NaN,7343084.0,1052801.0,943899.0,9339784.0
3,경유 하이브리드,NaN,NaN,18376.0,NaN,NaN,18376.0
4,LPG,454141.0,1132058.0,194059.0,24355.0,430.0,1805043.0
5,LPG 하이브리드,13062.0,NaN,NaN,NaN,NaN,13062.0
6,LNG,NaN,2.0,NaN,NaN,NaN,2.0
7,CNG,NaN,22511.0,4444.0,1.0,7.0,26963.0
8,CNG 하이브리드,275.0,17.0,NaN,NaN,NaN,292.0
9,전기,478797.0,NaN,NaN,NaN,NaN,478797.0


## 2\. 5등급 대기관리권역, 시도별 차량 현황

In [417]:
dfg['배출가스등급'].unique()

array(['3', '2', '1', '5', '4'], dtype=object)

In [418]:
g5 = dfg[dfg['배출가스등급'] == '5'].reset_index(drop=True)
g5.shape

(973967, 17)

In [419]:
g5.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index()

대기관리권역       시도     차대번호          
대기관리권역YN                        N         Y
0           그 외  강원특별자치도  39205.0       NaN
1           그 외  제주특별자치도  20063.0       NaN
2           남부권    광주광역시      NaN   21861.0
3           남부권     전라남도  38207.0   27852.0
4           동남권     경상남도  36574.0   50992.0
5           동남권     경상북도  54588.0   46225.0
6           동남권    대구광역시      NaN   43420.0
7           동남권    부산광역시      NaN   54144.0
8           동남권    울산광역시      NaN   20632.0
9           수도권      경기도   7904.0  189399.0
10          수도권    서울특별시      NaN  111270.0
11          수도권    인천광역시   1251.0   45499.0
12          중부권    대전광역시      NaN   21468.0
13          중부권  세종특별자치시      NaN    3566.0
14          중부권     전라북도  27612.0   22871.0
15          중부권     충청남도   2471.0   50963.0
16          중부권     충청북도   8937.0   26993.0

In [420]:
g5.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index().iloc[:, 2:].sum()

      대기관리권역YN
차대번호  N           236812.0
      Y           737155.0
dtype: float64

In [421]:
stat2 = g5.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index()
stat2.columns = ['대기관리권역', '시도', '대기관리권역_외', '대기관리권역_내']
stat2 = stat2[['대기관리권역', '시도', '대기관리권역_내', '대기관리권역_외']]
stat2

,대기관리권역,시도,대기관리권역_내,대기관리권역_외
0,그 외,강원특별자치도,NaN,39205.0
1,그 외,제주특별자치도,NaN,20063.0
2,남부권,광주광역시,21861.0,NaN
3,남부권,전라남도,27852.0,38207.0
4,동남권,경상남도,50992.0,36574.0
5,동남권,경상북도,46225.0,54588.0
6,동남권,대구광역시,43420.0,NaN
7,동남권,부산광역시,54144.0,NaN
8,동남권,울산광역시,20632.0,NaN
9,수도권,경기도,189399.0,7904.0


In [422]:
grp2 = pd.DataFrame()
percentage = 1
in_air_sum = 0
out_air_sum = 0
total = 0
for div in stat2['대기관리권역'].unique():
    temp = stat2[stat2['대기관리권역'] == div].reset_index(drop=True)
    temp['전체'] = temp.iloc[:, 2:].sum(axis=1)

    in_air_div_sum = temp['대기관리권역_내'].sum()
    out_air_div_sum = temp['대기관리권역_외'].sum()
    div_sum = temp['전체'].sum()
    temp.loc[temp.shape[0]] = [div, '소계', in_air_div_sum, out_air_div_sum, div_sum]

    in_air_sum += in_air_div_sum
    out_air_sum += out_air_div_sum
    total += div_sum
    temp.iloc[:, 2:] = temp.iloc[:, 2:].fillna(0)
    temp['대기관리권역_내_비율'] = temp['대기관리권역_내'] / temp['전체'] * percentage
    temp['대기관리권역_외_비율'] = temp['대기관리권역_외'] / temp['전체'] * percentage
    grp2 = pd.concat([grp2, temp], ignore_index=True)
    
grp2.loc[grp2.shape[0]] = ['합계', '합계', in_air_sum, out_air_sum, total, in_air_sum/total * percentage, out_air_sum/total * percentage]
grp2['전체비율'] = grp2['전체'] / total * percentage
grp2[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']] = grp2[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']].round(2)
grp2 = grp2[['대기관리권역', '시도', '전체', '전체비율', '대기관리권역_내', '대기관리권역_내_비율', '대기관리권역_외', '대기관리권역_외_비율']]

In [423]:
in_air_sum, out_air_sum

(737155.0, 236812.0)

In [424]:
grp2

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,그 외,강원특별자치도,39205.0,0.04,0.0,0.00,39205.0,1.00
1,그 외,제주특별자치도,20063.0,0.02,0.0,0.00,20063.0,1.00
2,그 외,소계,59268.0,0.06,0.0,0.00,59268.0,1.00
3,남부권,광주광역시,21861.0,0.02,21861.0,1.00,0.0,0.00
4,남부권,전라남도,66059.0,0.07,27852.0,0.42,38207.0,0.58
5,남부권,소계,87920.0,0.09,49713.0,0.57,38207.0,0.43
6,동남권,경상남도,87566.0,0.09,50992.0,0.58,36574.0,0.42
7,동남권,경상북도,100813.0,0.10,46225.0,0.46,54588.0,0.54
8,동남권,대구광역시,43420.0,0.04,43420.0,1.00,0.0,0.00
9,동남권,부산광역시,54144.0,0.06,54144.0,1.00,0.0,0.00


In [425]:
idf2 = pd.DataFrame({
    '대기관리권역':[
        '수도권', '수도권', '수도권', '수도권', 
        '중부권', '중부권', '중부권', '중부권', '중부권', '중부권', 
        '남부권', '남부권', '남부권',
        '동남권', '동남권', '동남권', '동남권', '동남권', '동남권',
        '그 외', '그 외', '그 외',
        '합계',
        ],
    '시도':[
        '서울특별시', '인천광역시', '경기도', '소계', 
        '대전광역시', '세종특별자치시', '충청북도', '충청남도', '전라북도', '소계', 
        '광주광역시', '전라남도', '소계',
        '부산광역시', '대구광역시', '울산광역시', '경상북도', '경상남도', '소계',
        '강원특별자치도', '제주특별자치도', '소계',
        '합계',
        ]
                     })
idf2

,대기관리권역,시도
0,수도권,서울특별시
1,수도권,인천광역시
2,수도권,경기도
3,수도권,소계
4,중부권,대전광역시
5,중부권,세종특별자치시
6,중부권,충청북도
7,중부권,충청남도
8,중부권,전라북도
9,중부권,소계


In [426]:
base2 = idf2.merge(grp2, on=['대기관리권역', '시도'], how='left')
base2

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,수도권,서울특별시,111270.0,0.11,111270.0,1.00,0.0,0.00
1,수도권,인천광역시,46750.0,0.05,45499.0,0.97,1251.0,0.03
2,수도권,경기도,197303.0,0.20,189399.0,0.96,7904.0,0.04
3,수도권,소계,355323.0,0.36,346168.0,0.97,9155.0,0.03
4,중부권,대전광역시,21468.0,0.02,21468.0,1.00,0.0,0.00
5,중부권,세종특별자치시,3566.0,0.00,3566.0,1.00,0.0,0.00
6,중부권,충청북도,35930.0,0.04,26993.0,0.75,8937.0,0.25
7,중부권,충청남도,53434.0,0.05,50963.0,0.95,2471.0,0.05
8,중부권,전라북도,50483.0,0.05,22871.0,0.45,27612.0,0.55
9,중부권,소계,164881.0,0.17,125861.0,0.76,39020.0,0.24


## 3\. 전체 차량 대기관리권역, 시도별, 차량 현황

In [427]:
dfg.shape

(24767342, 17)

In [428]:
dfg.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index().iloc[:, 2:].sum()

      대기관리권역YN
차대번호  N            3814717.0
      Y           20952625.0
dtype: float64

In [429]:
stat3 = dfg.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index()
stat3.columns = ['대기관리권역', '시도', '대기관리권역_외', '대기관리권역_내']
stat3 = stat3[['대기관리권역', '시도', '대기관리권역_내', '대기관리권역_외']]
stat3

,대기관리권역,시도,대기관리권역_내,대기관리권역_외
0,그 외,강원특별자치도,NaN,833127.0
1,그 외,제주특별자치도,NaN,660359.0
2,남부권,광주광역시,696673.0,NaN
3,남부권,전라남도,586804.0,596181.0
4,동남권,경상남도,1322320.0,529094.0
5,동남권,경상북도,921329.0,561708.0
6,동남권,대구광역시,1206483.0,NaN
7,동남권,부산광역시,1429492.0,NaN
8,동남권,울산광역시,573980.0,NaN
9,수도권,경기도,6105741.0,126477.0


In [430]:
grp3 = pd.DataFrame()
percentage = 1
in_air_sum = 0
out_air_sum = 0
total = 0
for div in stat3['대기관리권역'].unique():
    temp = stat3[stat3['대기관리권역'] == div].reset_index(drop=True)
    temp['전체'] = temp.iloc[:, 2:].sum(axis=1)

    in_air_div_sum = temp['대기관리권역_내'].sum()
    out_air_div_sum = temp['대기관리권역_외'].sum()
    div_sum = temp['전체'].sum()
    temp.loc[temp.shape[0]] = [div, '소계', in_air_div_sum, out_air_div_sum, div_sum]

    in_air_sum += in_air_div_sum
    out_air_sum += out_air_div_sum
    total += div_sum
    temp.iloc[:, 2:] = temp.iloc[:, 2:].fillna(0)
    temp['대기관리권역_내_비율'] = temp['대기관리권역_내'] / temp['전체'] * percentage
    temp['대기관리권역_외_비율'] = temp['대기관리권역_외'] / temp['전체'] * percentage
    grp3 = pd.concat([grp3, temp], ignore_index=True)
    
grp3.loc[grp3.shape[0]] = ['합계', '합계', in_air_sum, out_air_sum, total, in_air_sum/total * percentage, out_air_sum/total * percentage]
grp3['전체비율'] = grp3['전체'] / total * percentage
grp3[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']] = grp3[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']].round(2)
grp3 = grp3[['대기관리권역', '시도', '전체', '전체비율', '대기관리권역_내', '대기관리권역_내_비율', '대기관리권역_외', '대기관리권역_외_비율']]

In [431]:
in_air_sum, out_air_sum

(20952625.0, 3814717.0)

In [432]:
grp3

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,그 외,강원특별자치도,833127.0,0.03,0.0,0.00,833127.0,1.00
1,그 외,제주특별자치도,660359.0,0.03,0.0,0.00,660359.0,1.00
2,그 외,소계,1493486.0,0.06,0.0,0.00,1493486.0,1.00
3,남부권,광주광역시,696673.0,0.03,696673.0,1.00,0.0,0.00
4,남부권,전라남도,1182985.0,0.05,586804.0,0.50,596181.0,0.50
5,남부권,소계,1879658.0,0.08,1283477.0,0.68,596181.0,0.32
6,동남권,경상남도,1851414.0,0.07,1322320.0,0.71,529094.0,0.29
7,동남권,경상북도,1483037.0,0.06,921329.0,0.62,561708.0,0.38
8,동남권,대구광역시,1206483.0,0.05,1206483.0,1.00,0.0,0.00
9,동남권,부산광역시,1429492.0,0.06,1429492.0,1.00,0.0,0.00


In [433]:
base3 = idf2.merge(grp3, on=['대기관리권역', '시도'], how='left')
base3

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,수도권,서울특별시,3076727.0,0.12,3076727.0,1.00,0.0,0.00
1,수도권,인천광역시,1607098.0,0.06,1593783.0,0.99,13315.0,0.01
2,수도권,경기도,6232218.0,0.25,6105741.0,0.98,126477.0,0.02
3,수도권,소계,10916043.0,0.44,10776251.0,0.99,139792.0,0.01
4,중부권,대전광역시,693839.0,0.03,693839.0,1.00,0.0,0.00
5,중부권,세종특별자치시,190487.0,0.01,190487.0,1.00,0.0,0.00
6,중부권,충청북도,899641.0,0.04,780165.0,0.87,119476.0,0.13
7,중부권,충청남도,1190767.0,0.05,1159398.0,0.97,31369.0,0.03
8,중부권,전라북도,959015.0,0.04,615404.0,0.64,343611.0,0.36
9,중부권,소계,3933749.0,0.16,3439293.0,0.87,494456.0,0.13


## 4\. 무공해차 대기관리권역, 시도별 현황

In [434]:
dfg['연료'].unique()

array(['경유', '휘발유', '휘발유 하이브리드', '전기', 'LPG', '수소', '경유 하이브리드', 'CNG',
       '기타연료', 'LPG 하이브리드', 'CNG 하이브리드', 'LNG', '알코올'], dtype=object)

In [435]:
dfg.shape

(24767342, 17)

In [436]:
zev = dfg[(dfg['연료'] == '전기') | (dfg['연료'] == '수소')].reset_index(drop=True)
zev.shape

(511706, 17)

In [437]:
zev.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index().iloc[:, 2:].sum()

      대기관리권역YN
차대번호  N            98280.0
      Y           413426.0
dtype: float64

In [438]:
stat4 = zev.groupby(['대기관리권역', '대기관리권역YN', '시도']).agg({'차대번호':'count'}).unstack('대기관리권역YN').reset_index()
stat4.columns = ['대기관리권역', '시도', '대기관리권역_외', '대기관리권역_내']
stat4 = stat4[['대기관리권역', '시도', '대기관리권역_내', '대기관리권역_외']]
stat4

,대기관리권역,시도,대기관리권역_내,대기관리권역_외
0,그 외,강원특별자치도,NaN,19116.0
1,그 외,제주특별자치도,NaN,36929.0
2,남부권,광주광역시,12071.0,NaN
3,남부권,전라남도,11680.0,10635.0
4,동남권,경상남도,25175.0,8379.0
5,동남권,경상북도,14453.0,10256.0
6,동남권,대구광역시,28501.0,NaN
7,동남권,부산광역시,31045.0,NaN
8,동남권,울산광역시,9574.0,NaN
9,수도권,경기도,102424.0,2479.0


In [439]:
grp4 = pd.DataFrame()
percentage = 1
in_air_sum = 0
out_air_sum = 0
total = 0
for div in stat4['대기관리권역'].unique():
    temp = stat4[stat4['대기관리권역'] == div].reset_index(drop=True)
    temp['전체'] = temp.iloc[:, 2:].sum(axis=1)

    in_air_div_sum = temp['대기관리권역_내'].sum()
    out_air_div_sum = temp['대기관리권역_외'].sum()
    div_sum = temp['전체'].sum()
    temp.loc[temp.shape[0]] = [div, '소계', in_air_div_sum, out_air_div_sum, div_sum]

    in_air_sum += in_air_div_sum
    out_air_sum += out_air_div_sum
    total += div_sum
    temp.iloc[:, 2:] = temp.iloc[:, 2:].fillna(0)
    temp['대기관리권역_내_비율'] = temp['대기관리권역_내'] / temp['전체'] * percentage
    temp['대기관리권역_외_비율'] = temp['대기관리권역_외'] / temp['전체'] * percentage
    grp4 = pd.concat([grp4, temp], ignore_index=True)
    
grp4.loc[grp4.shape[0]] = ['합계', '합계', in_air_sum, out_air_sum, total, in_air_sum/total * percentage, out_air_sum/total * percentage]
grp4['전체비율'] = grp4['전체'] / total * percentage
grp4[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']] = grp4[['대기관리권역_내_비율', '대기관리권역_외_비율', '전체비율']].round(2)
grp4 = grp4[['대기관리권역', '시도', '전체', '전체비율', '대기관리권역_내', '대기관리권역_내_비율', '대기관리권역_외', '대기관리권역_외_비율']]

In [440]:
in_air_sum, out_air_sum

(413426.0, 98280.0)

In [441]:
grp4

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,그 외,강원특별자치도,19116.0,0.04,0.0,0.00,19116.0,1.00
1,그 외,제주특별자치도,36929.0,0.07,0.0,0.00,36929.0,1.00
2,그 외,소계,56045.0,0.11,0.0,0.00,56045.0,1.00
3,남부권,광주광역시,12071.0,0.02,12071.0,1.00,0.0,0.00
4,남부권,전라남도,22315.0,0.04,11680.0,0.52,10635.0,0.48
5,남부권,소계,34386.0,0.07,23751.0,0.69,10635.0,0.31
6,동남권,경상남도,33554.0,0.07,25175.0,0.75,8379.0,0.25
7,동남권,경상북도,24709.0,0.05,14453.0,0.58,10256.0,0.42
8,동남권,대구광역시,28501.0,0.06,28501.0,1.00,0.0,0.00
9,동남권,부산광역시,31045.0,0.06,31045.0,1.00,0.0,0.00


In [442]:
base4 = idf2.merge(grp4, on=['대기관리권역', '시도'], how='left')
base4

,대기관리권역,시도,전체,전체비율,대기관리권역_내,대기관리권역_내_비율,대기관리권역_외,대기관리권역_외_비율
0,수도권,서울특별시,69657.0,0.14,69657.0,1.00,0.0,0.00
1,수도권,인천광역시,35087.0,0.07,34926.0,1.00,161.0,0.00
2,수도권,경기도,104903.0,0.21,102424.0,0.98,2479.0,0.02
3,수도권,소계,209647.0,0.41,207007.0,0.99,2640.0,0.01
4,중부권,대전광역시,17638.0,0.03,17638.0,1.00,0.0,0.00
5,중부권,세종특별자치시,4161.0,0.01,4161.0,1.00,0.0,0.00
6,중부권,충청북도,20102.0,0.04,17484.0,0.87,2618.0,0.13
7,중부권,충청남도,22870.0,0.04,22280.0,0.97,590.0,0.03
8,중부권,전라북도,19474.0,0.04,12357.0,0.63,7117.0,0.37
9,중부권,소계,84245.0,0.16,73920.0,0.88,10325.0,0.12


### [출력] 통계
- 전체 차량 등급, 연료별 차량 현황
- 5등급 대기관리권역, 시도별 차량 현황
- 전체 차량 대기관리권역, 시도별 차량 현황
- 무공해차(전기, 수소) 대기관리권역, 시도별 차량 현황

In [443]:
today_date = datetime.today().strftime("%Y%m%d_%p_%I%M%S")
today_date

'20230904_PM_024801'

In [444]:
# 0.3s
with pd.ExcelWriter(os.path.join(an_fold, f'통계_챠량현황({today_date}).xlsx')) as writer:
    base1.to_excel(writer, sheet_name='전체_등급_연료별', index=False)
    base2.to_excel(writer, sheet_name='5등급_대기관리권역별', index=False)
    base3.to_excel(writer, sheet_name='전체_대기관리권역별', index=False)
    base4.to_excel(writer, sheet_name='무공해_대기관리권역별', index=False)

# [로드] 등록&제원 정보 2022.12.28 기준 

In [4]:
# about 8m 22s
df = pd.DataFrame()
for i in tqdm(range(1, 6)):
    file_name = f'[ODS][G{i}]등록정보&제원정보(말소제거)(2022.12.28).csv'
    temp = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
    temp = temp.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
    df = pd.concat([df, temp], ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [08:36<00:00, 103.25s/it]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25404385 entries, 0 to 25404384
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [6]:
df.shape

(25404385, 49)

In [7]:
len(df['차대번호'].unique())

25404384

In [10]:
file_name = '법정동코드_2022.10.14.xlsx'
code = pd.read_excel(os.path.join(fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


# 코드 마지막